<a href="https://colab.research.google.com/github/llu13701/cm1_python_test/blob/main/CM1_DevQuickTest_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Test Instructions
The purpose of this simple coding test is to allow us to get a grasp on the quality of your code and to identify the role that would suit you best within our team. You don't NEED to be able to answer all of the items, but the ones that you do answer need to be correct. Feel free to use Google or any other tools that you prefer to complete these tasks.

Install any needed third-party libraries below this block. Please install the minimum amount of libraries you need.

!pip install whatever_you_need

In [359]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly

import os
import discord
from dotenv import load_dotenv
from typing import Dict, List, Any

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain import LLMChain
from langchain.memory import ChatMessageHistory
from langchain import chat_models


### Task 1 (Data manipulation / Visualization)
Download the file: https://drive.google.com/file/d/1R_M3xI3b_BqAN_xWLqJDneg273X_sok5/view?usp=sharing

In `message_data.csv` there's two columns, one corresponds to the unique id of someone sending a message and the other one to when they sent it.

Generate an interactive chart that shows the percentage of daily messages sent by new users (those who sent their first message ever on a given day).

In [402]:
# ~20 seconds

message_data_file = 'message_data.csv'

message_data = pd.read_csv(message_data_file, parse_dates=["message_time"])
message_data['message_time'] = pd.to_datetime(message_data['message_time'], format="%Y-%m-%d %H:%M:%S.%f000%z", errors="coerce")

# get rid of bad dates - save for later
bad_dates = message_data.loc[message_data['message_time'].isnull()]
message_data = message_data.loc[message_data['message_time'].notnull()]

# date field for aggregation
message_data['date'] = pd.to_datetime(message_data['message_time'].dt.date)

#
daily_message_count = message_data.groupby(['date'])['message_id'].count().reset_index(name='message_count')
daily_message_count['date'] = pd.to_datetime(daily_message_count['date'])

# main df
first_messages = message_data.groupby(['author_id'])['message_time'].min().reset_index(name='first_message')
first_messages['date'] = pd.to_datetime(first_messages['first_message'].dt.date)

# get count of messages each user sent on first day
first_messages['count'] = 0

for d, a in zip(first_messages['date'], first_messages['author_id']):

    c = message_data[ (message_data['date'] == d) & (message_data['author_id'] == a) ].groupby(['author_id'])['message_id'].count().reset_index(name='count').get('count')[0]
    
    first_messages.loc[(first_messages['date'] == d) & (first_messages['author_id'] == a), 'count'] = c

    first_messages['pct'] = 0
first_messages['days_total'] = 0

for d, a, c in zip(first_messages['date'], first_messages['author_id'], first_messages['count']):
    for dc in daily_message_count.loc[daily_message_count['date'] == d, 'message_count']:
        # set each day's total message count
        first_messages.loc[(first_messages['date'] == d) & (first_messages['author_id'] == a), 'days_total'] = dc
        # set each day's new user message count%
        first_messages.loc[(first_messages['date'] == d) & (first_messages['author_id'] == a), 'pct'] = (c / dc) * 100


# count all new users' messages on their first day chatting
a = first_messages.groupby('date')['count'].sum().reset_index(name='new_user_messages_count')
a['date'] = pd.to_datetime(a['date'])
b = daily_message_count.merge(a)
b['pct'] = b['new_user_messages_count'] / b['message_count'] * 100
print(b.head())  # passes a sniff test


        date  message_count  new_user_messages_count         pct
0 2022-05-01              1                        1  100.000000
1 2022-05-02              5                        3   60.000000
2 2022-05-03            830                      786   94.698795
3 2022-05-04           2937                     1323   45.045965
4 2022-05-05           1384                      274   19.797688


Executing <Task pending name='Task-4' coro=<Kernel.dispatch_queue() running at /home/zeebrow/Documents/work/communityone-coding-challenge/cm1_python_test/venv/lib/python3.10/site-packages/ipykernel/kernelbase.py:516> wait_for=<Future pending cb=[Task.task_wakeup()] created at /home/zeebrow/Documents/work/communityone-coding-challenge/cm1_python_test/venv/lib/python3.10/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/zeebrow/Documents/work/communityone-coding-challenge/cm1_python_test/venv/lib/python3.10/site-packages/tornado/ioloop.py:685] created at /usr/lib/python3.10/asyncio/tasks.py:636> took 12.920 seconds


In [405]:

fig = plotly.graph_objects.Figure(data=[
        plotly.graph_objects.Bar(
            name='All Messages',
            y=b['message_count'],
            x=b['date'],
            customdata=b['pct'].array,
            hovertemplate="<br>".join([
                "%{x}",
                "<b>%{customdata:.1f}% New User Messages</b>",
                "<b>Total Messages Sent:</b> %{y}"
            ]),
        ),
        plotly.graph_objects.Bar(
            name='New User Messages',
            y=b['new_user_messages_count'],
            x=b['date'],
            customdata=b['pct'],
            hovertemplate="<br>".join([
                "%{x}",
                "<b>%{customdata:.1f}% New User Messages</b>",
                "<b>New User Messages:</b> %{y}"
            ]),
        )
])
fig.update_layout(barmode='overlay', title_text="New User Message Volume by Date - Hover for Percentage")
fig.update_yaxes(title='Message Count')
fig.update_xaxes(title='Date')
# fig.update_traces(hovertemplate=hovertemp)
fig.show()


### Task 2 (Django)
**To take the django portion of this test go to: https://github.com/llu13701/cm1_python_test and follow the instructions.**

<https://github.com/zeebrow/django_reddit.git>

### 1 name and date

<img src='djangopics/1-name-date.png'>

### 2 show url and link to comments

<img src='djangopics/2-link-to-comments-show-link.png'>

### 3 profile with submissions and comments

<img src='djangopics/3-profile-submissions-comments.png'>

### 4 edit dialog

<img src='djangopics/4-1-needs-edit.png'>
<img src='djangopics/4-2-edited.png'>

### 5 video of edits

<https://youtu.be/fuyMKXz_H7Q>

### Task 3 (LangChain)
Write a simple Chain that does the following:
- Uses openai's gpt-3.5-turbo model.
- Remembers the last 2 message exchanges, as well as the system message.
- Greets people and tells a joke about their name.

In [ ]:
load_dotenv()


chat = chat_models.ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.8
)

SPECIAL_SAUCE = "Be kid-friendly."
# SPECIAL_SAUCE = "Be absolutely brutal. Show no mercy, nor apologies."
# SPECIAL_SAUCE = "Make sure the joke would be awesome to tell during a job interview."
system_prompt_template = "Greet the discord users with a joke about their name."
system_prompt_template = " ".join([
    system_prompt_template, 
    SPECIAL_SAUCE, 
    "Do not respond with questions.",
])
human_prompt_template = "{username} has joined."


system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        template=system_prompt_template,
        input_variables=[]
    )
)

history = ChatMessageHistory()
name = input("what is your discord name? ")

human_prompt_username = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=human_prompt_template,
        input_variables=['username']
    )
)

history.add_message(system_prompt)
history.add_message(human_prompt_username.format(username=name))

chat_prompt = ChatPromptTemplate.from_messages(history.messages)
chain = LLMChain(llm=chat, prompt=chat_prompt)

cr = chain.run({'username': name})
print(cr)
history.add_ai_message(cr)

response_text = input("Tell the chatbot what you thought of the joke! ")
human_prompt_response = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="{text}",
        input_variables=['text']
    )
)

history.add_message(human_prompt_response.format(text=response_text))

chat_prompt = ChatPromptTemplate.from_messages(history.messages)
chain = LLMChain(llm=chat, prompt=chat_prompt)
cr = chain.run({
    'username': name,
    'text' : response_text
})
print(cr)

### Task 4 (Discord API)

Write a very simple discord bot that says "Hello {user_name}!" to every new incoming message in a discord server.


In [ ]:
load_dotenv()

# Bot:
intents = discord.Intents.default()
intents.message_content = True


TOKEN = os.getenv('DISCORD_TOKEN')


client = discord.Client(intents=intents)

@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
    if message.author == client.user:
        return
    await message.channel.send(f"Hello, {message.author.global_name}!")
    return

client.run(TOKEN)

### Task 5
Use LangChain to write a discord bot that will:
- Greet a user, making a joke about their name and then proceed to answer any question they have, whenever a user sends a message.
- Use async with langchain chain calls.


In [ ]:

load_dotenv()

# Session:
CHAT_CACHE: Dict[str, List[str]] = {}

chat = chat_models.ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.8
)

SPECIAL_SAUCE = "Be kid-friendly."
# SPECIAL_SAUCE = "They are a priest."
# SPECIAL_SAUCE = "Roast them."
# SPECIAL_SAUCE = "Be absolutely brutal. Show no mercy, nor apologies."
# SPECIAL_SAUCE = "Make sure the joke would be awesome to tell during a job interview."
# SPECIAL_SAUCE = ""
system_prompt_template = "Greet the discord user named {username} with a joke about their name."
system_prompt_template = " ".join([
    system_prompt_template, 
    SPECIAL_SAUCE, 
    "Do not respond with questions.",
])
human_prompt_template = "{text}"

def start_chat(who: str, text: str) -> str:
    global CHAT_CACHE
    CHAT_CACHE[who] = []
    
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt_template)
    human_prompt = HumanMessagePromptTemplate.from_template(human_prompt_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
#     chain = LLMChain(llm=chat, prompt=chat_prompt)
#     resp = chain.run(username=who, text=text)
    msgs = chat_prompt.format_prompt(username=who, text=text).to_messages()
    resp = chat(msgs)
    for msg in msgs:
        CHAT_CACHE[who].append(msg)
    CHAT_CACHE[who].append(resp)
    return resp.content
    

def respond_to_model(who: str, text: str):
    global CHAT_CACHE
    human_prompt = HumanMessagePromptTemplate.from_template(human_prompt_template)
    past_msgs = CHAT_CACHE[who]
    print(past_msgs)

    chat_prompt = ChatPromptTemplate.from_messages([*past_msgs, human_prompt])
    msgs = chat_prompt.format_prompt(text=text).to_messages()
    resp = chat(msgs)
    for msg in msgs:
        CHAT_CACHE[who].append(msg)
    CHAT_CACHE[who].append(resp)
    return resp.content


def cleanup(who: str):
    print(f"cleaning up session with {who}")
    global CHAT_CACHE
    del CHAT_CACHE[who]



# Bot:
intents = discord.Intents.default()
intents.message_content = True


TOKEN = os.getenv('DISCORD_TOKEN')


client = discord.Client(intents=intents)

@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
    if message.author == client.user:
        return
    user = message.author.global_name
    text = message.content
    if user in CHAT_CACHE.keys():
        ai_response = respond_to_model(user, text)
        await message.channel.send(ai_response)
        cleanup(user)
    else:
        ai_response = start_chat(user, text)
        await message.channel.send(ai_response)
    return

client.run(TOKEN)